In [1]:
import numpy as np
import nltk
import matplotlib.pyplot as plt
import pandas as pd

#data scaling
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split , KFold, cross_val_score, LeaveOneOut
from sklearn.naive_bayes import MultinomialNB

import spacy

In [2]:
# preprocessing
data = pd.read_csv("TrainingSet1.csv")

print( data )

#temporaire pour moins de données
#data = data.head()

intentions = data["intention"].unique()
data["intention"] = data["intention"].replace({"arrosage" : np.where( intentions == 'arrosage'),
                                              "soleil": np.where( intentions == 'soleil'), 
                                              "tailler" : np.where( intentions == 'tailler'),
                                              "temperature" : np.where( intentions == 'temperature')
                                              })

                                             sentence    intention
0        Est ce que j'ai besoin d'arroser ma plante ?     arrosage
1                   Quand dois-je arroser ma plante ?     arrosage
2                         Dois-je arroser ma plante ?     arrosage
3                 Comment puis-je arroser ma plante ?     arrosage
4              Ma plante a besoin de beaucoup d'eau ?     arrosage
..                                                ...          ...
84                     dois je réchauffer ma plante ?  temperature
85                                refroidir la plante  temperature
86              comment puis-je refroidir ma plante ?  temperature
87  garder sa plante entre combien et combien de d...  temperature
88      à combien de degrés dois-je garder ma plante?  temperature

[89 rows x 2 columns]


In [3]:
y = list(data["intention"])

In [4]:
liste = data["sentence"].tolist()
j=0;

#dictionnaire des mots connus
words = list()
allowed_pos = ['VERB', 'NOUN', 'PROPN', 'ADJ']

# process sentences
for i in data["sentence"]:
    # convert all letters to lower case
    i = i.lower()
    i = i.replace('-', ' ')

    nlp_fr = spacy.load('fr_core_news_sm')
    tokens = nlp_fr(i)

    new_sentence = ''
    for token in tokens:
        if ( allowed_pos.count(token.pos_) > 0 ):
            if (token.lemma_ != 'plante' and token.lemma_ != 't'): 
                new_sentence += str(token.lemma_) + ' '
                words.append(str(token.lemma_))
        
    liste[j] = new_sentence;
    j += 1

In [5]:
words = list(words)

In [6]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(liste).todense() #renvoie le bag of words

In [7]:
loo = LeaveOneOut()
loo.get_n_splits(X)

89

In [8]:
compt_naive_bayes = 0
compt_svm = 0

for train_index, test_index in loo.split(X):  #les differents jeux de données
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train = []
    y_train = []
    for index in train_index:
        X_train+=X[index].tolist()
        y_train.append(y[index])
    X_test = X[test_index].tolist()
    y_test = y[int(test_index)]
    
    clf_svm = svm.SVC(kernel='linear')
    clf_bayes = MultinomialNB(alpha=0.01)
    clf_svm.fit(X_train, y_train)
    clf_bayes.fit(X_train, y_train)
    
    if int(clf_bayes.predict(X_test))==y_test:  # si prediction naive bayes correcte
        compt_naive_bayes += 1 
        
    if int(clf_svm.predict(X_test))==y_test:  # si prediction svm correcte
        compt_svm += 1 

In [9]:
print("Précision & Recall Naive Bayes :", compt_naive_bayes/len(X)) 
print("Précision & Recall SVM :", compt_svm/len(X))

Précision & Recall Naive Bayes : 0.7415730337078652
Précision & Recall SVM : 0.8202247191011236


In [10]:
# Exportons les deux modèles...

from joblib import dump, load
dump(clf_bayes, 'clf_bayes.joblib') 
dump(clf_svm, 'clf_svm.joblib')

['clf_svm.joblib']

In [13]:
# Avant de les charger à nouveau
clf_bayes = load('clf_bayes.joblib') 
clf_svm = load('clf_svm.joblib') 

In [14]:
# Test with a simple sentence
s = "avoir besoin soleil ?"

# Pre-processing
s = s.lower()

nlp_fr = spacy.load('fr_core_news_sm')
tokens = nlp_fr(s)

words = list()
                
# Lemmatize
for token in tokens:
    if ( allowed_pos.count(token.pos_) > 0 ):
        if (token.lemma_ != 'plante' and token.lemma_ != 't'): 
            words.append(str(token.lemma_))

           
j = 0;
vector = vectorizer.get_feature_names()

# Create vector
for word in vector:
    vector[j] = words.count(word);           
    j += 1



p = clf_svm.predict([vector])
print(intentions[p])

['soleil']
